In [1]:
import os
import nltk
import math
import random
import collections

In [2]:
parent = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
parent = parent.split('/')
parent.remove(parent[-1])
parent = '/'.join(parent)
categories = ['entertainment', 'sports', 'fun', 'games', 'weather', 'science', 'technology', 'politics']

In [3]:
def fileids(category):
    path = os.path.join(parent, 'corpus', 'processed', category)
    return os.listdir(path)

def words(file):
    f = open(file, 'r', encoding='ISO-8859-1').read().strip()
    sents = [sent.split(' ') for sent in f.split('\n')]
    words = [word for sent in sents for word in sent if len(word) > 0]
    return words

In [7]:
documents, total_words = [], []
for category in categories:
    for fileid in fileids(category):
        path = os.path.join(parent, 'corpus', 'processed',
                            category, fileid)
        w = words(path)
        documents.append((w, category))
        total_words.extend(w)
print('Word count:', len(total_words))
#random.shuffle(documents)
print(total_words[:20])

Word count: 4110443
['\x00\x00\x00\x01Bud1\x00\x00\x10\x00\x00\x00\x08\x00\x00\x00\x10\x00\x00\x00\x00%\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [5]:
def features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [6]:
def display(num):
    return '{0:.2f}'.format(num)

In [7]:
from nltk.classify import accuracy
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.linear_model import LogisticRegression

nb_avg = 0
svc_avg = 0
lr_avg = 0

iters = 3

for i in range(iters):
    
    print('Running iteration {}'.format(i + 1))
    
    random.shuffle(documents)
    
    all_words = set(total_words)
    word_features = list(all_words)
    random.shuffle(word_features)
    word_features = word_features[:5000]
    #print('Vocab size:', len(all_words))
    
    feature_sets = [(features(d), c) for (d, c) in documents]
    #print('Documents:', len(feature_sets))
    cutoff = math.ceil(len(feature_sets) * 0.7)
    train_set, test_set = feature_sets[:cutoff], feature_sets[cutoff:]

    classifier = SklearnClassifier(MultinomialNB())
    classifier.train(train_set)
    acc = accuracy(classifier, test_set)
    nb_avg += acc

    classifier = SklearnClassifier(NuSVC())
    classifier.train(train_set)
    acc = accuracy(classifier, test_set)
    svc_avg += acc

    classifier = SklearnClassifier(LogisticRegression())
    classifier.train(train_set)
    acc = accuracy(classifier, test_set)
    lr_avg += acc

print()
print('MultinomialNB: {}%'.format(display((nb_avg / iters) * 100)))
print('NuSVC: {}%'.format(display((svc_avg / iters) * 100)))
print('LogisticRegression: {}%'.format(display((lr_avg / iters) * 100)))

Running iteration 1
Running iteration 2
Running iteration 3

MultinomialNB: 88.19%
NuSVC: 89.58%
LogisticRegression: 94.44%
